In [1]:
import sys
sys.path.append("../../")

from itops.storage.azure_blob.azure_blob_helper import AzureBlobHelper
from itops.storage.azure_blob.csv_helper import CSVHelper
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from itops.db.mysql.mysqlhelper import MySQLHelper
from itops.llm.azureopenaimanager.azure_open_ai_helper import AzureOpenAIManager
from itops.config.configs import CONFIGS
mysql_helper = MySQLHelper(CONFIGS.HOST,
                           CONFIGS.USERNAME_MYSQL,
                           CONFIGS.PASSWORD, "itops")

c:\Ambarish\itops\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


True


In [4]:
# Read a Parquet file into a DataFrame
df = pd.read_parquet('01.parquet')

print(df.columns, len(df))

Index(['ID_request', 'Text', 'Category', 'Solution', 'Date_request_recieved',
       'Date_request_solved', 'ID_agent', 'themes', 'embeddings', 'CLUSTERS',
       'CLUSTER_NAMES'],
      dtype='object') 118


In [ ]:
description_column_name = "Text"
MODEL_NAME = "all-MiniLM-L6-v2"
CATEGORY = "ITSM1"
run_name = "RUN004"
user_input = "Please extract the theme of the provided context in Maximum 5 words"

In [ ]:
FILENAME = 'ITSM_Data-RUN001-ITSM1-RUN002-ITSM1.csv'
FILENAME = 'ITSM_Data-RUN001-ITSM1-DEVICE_RUN_001-ITSM1.csv'
FILENAME = 'ITSM_Data-RUN001-ITSM1-RUN004-ITSM1.csv'
CONTAINER_NAME = 'itops'
ACCOUNT_NAME = 'stgtxtsql'
blob_helper = AzureBlobHelper(account_name= ACCOUNT_NAME,
                              container_name= CONTAINER_NAME,
                              account_key=CONFIGS.AZURE_BLOB_STORAGE_KEY)
csv_helper = CSVHelper(azure_blob_helper=blob_helper)
df = csv_helper.read_csv(FILENAME)

In [ ]:
df.columns

In [ ]:
df["themes"].unique()

In [ ]:
len(df["CLUSTER_NAMES"].unique())

In [ ]:
df["CLUSTERS"].unique()

In [ ]:
df["CLUSTER_NAMES"].unique()

In [ ]:
df["CLUSTER_NAMES"].value_counts()

In [ ]:
df_dropped = df.drop('embeddings', axis=1)
df_dropped = df_dropped.drop('CLUSTERS', axis=1)
df_dropped = df_dropped.drop('CLUSTER_NAMES', axis=1)

In [ ]:
df_dropped.columns

In [ ]:
def get_embedding_query_vector(query,model_name):
        """Get the vector of the query

        Args:
            query (string): user input

        Returns:
            _type_: vector of the query
        """
        model = SentenceTransformer(model_name)
        query_vector = model.encode(query)
        return query_vector

In [ ]:
MODEL_NAME = "all-MiniLM-L6-v2"

In [ ]:
def generate_embedding_dataset(df):
    embedding_list = []
    for i in range(len(df)):
        content = df.iloc[i]["themes"]
        embedding = get_embedding_query_vector(content,MODEL_NAME)
        embedding_list.append(embedding)
        print(f"Completed EMBEDDING  {i+1} ROW")

    df["embeddings"] = embedding_list
    return df

In [ ]:
df_new = generate_embedding_dataset(df_dropped)

In [ ]:
# Save the DataFrame to a Parquet file
df_new.to_parquet('PARQUET001.parquet', engine='pyarrow', compression='snappy')

print("DataFrame saved to Parquet file.")

In [ ]:
# Read the DataFrame back from the Parquet file
loaded_df = pd.read_parquet('PARQUET001.parquet')

In [ ]:
num_clusters = 6

In [ ]:
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
embedding_array = np.array(loaded_df["embeddings"].tolist())
kmeans.fit(embedding_array)
labels = kmeans.labels_
loaded_df["CLUSTERS"] = labels

In [ ]:
loaded_df["CLUSTERS"].value_counts()